In [7]:
import numpy as np
import pandas as pd

In [2]:
def load_data(path):
    data = np.loadtxt(path, delimiter=' ')
    return data


In [217]:
class Node():
    def __init__(self, attribute, threshold,left = None, right= None,label = None,info_gain = 0):
        self.label = label
        self.left = left
        self.right = right
        self.attribute = attribute
        self.threshold = threshold
        self.info_gain = info_gain

In [231]:
class Tree():
    def __init__(self, data):
        self.data = data
        self.root = self.build_tree(self.data)
    
    
    def best_attribute(self,data):
        max = float('-inf')
        attribute = 0
        threshold = 0
        
        for i in range(data.shape[1]-1):
            thresh = self.best_threshold(data, i)
            gain = self.info_gain(data, i, thresh)
            if gain > max:
                max = gain
                attribute = i
                threshold = thresh
                
        return attribute, threshold,gain
    
    def best_threshold(self, data, index):
        
        list = data[:,index]
        max = float('-inf')
        threshold = 0
        
        for i in range(len(list)):
            thresh = list[i]
            gain = self.info_gain(data, index, thresh)
            if gain > max:
                max = gain
                threshold = thresh
                
        return threshold
    
    
    def info_gain(self, data, index, threshold):
        
        left, right = self.split(data, index, threshold)
        left_entropy = self.entropy(left)
        right_entropy = self.entropy(right)
        entropy = self.entropy(data)
        left_num =  len(left)
        right_num = len(right)
        n = len(data)
        return entropy - (left_entropy*(left_num/n) + right_entropy*(right_num/n))
    
    def split(self, data, index, threshold):
        
        left = []
        right = []
        for i in range(len(data)):
            if data[i][index] < threshold:
                left.append(data[i])
            else:
                right.append(data[i])
                
        return left, right
       
    
    def entropy(self, data):
            # print(len(data))
            count = 0
            for i in range(len(data)):
                if data[i][-1] == 1:
                    count += 1
            
            if count == 0:
                return 0
            
            p = count / len(data)
            q = 1 - p
            
            if p == 0 or q == 0:
                return 0
            else:
                return -p*np.log2(p) - q*np.log2(q)
            
    def build_tree(self, data):
       
        
        one  = len(data[data[:,-1] == 1])
        zero = len(data[data[:,-1] == 0])
        if one == 0 or zero == 0:
            if zero == 0:
                return Node(None,None,label=1)
            else:
                return Node(None,None,label=0)
        
        attribute, threshold, gain = self.best_attribute(data)
        left, right = self.split(data, attribute, threshold)
        
        
        node = Node(attribute, threshold,info_gain=gain)
        node.left = self.build_tree(np.array(left))
        node.right = self.build_tree(np.array(right))
        return node
    
    def predict(self, data):
        
        node = self.root
        
        while node.left != None and node.right != None:
            if data[node.feature] < node.value:
                node = node.left
            else:
                node = node.right
                
        return node.label
    
    def accuracy(self, data):
        
        count = 0
        for i in range(len(data)):
            if self.predict(data[i]) == data[i][-1]:
                count += 1
                
        return count / len(data)
    
    def print_tree(self):
        
        self.print_node(self.root)
        
    def print_node(self, node, depth=0, branch=''):
        if node is None:
            return

        indent = "    " * depth

        if depth > 0:
            if branch == "left":
                print(indent + "├── ", end='')
            else:
                print(indent + "└── ", end='')

        if node.label is not None:
            print(f"Leaf Node: Predicted Label = {node.label}")
        else:
            print(f"Split on Attribute {node.attribute} with Threshold {node.threshold} Information Gain = {node.info_gain}")
            self.print_node(node.left, depth+1, branch="left")
            self.print_node(node.right, depth+1, branch="right")
    

In [236]:
data = load_data('E:\Classes\CS760\HW2\Homework 2 data\D2.txt')


In [237]:
dc = Tree(data)
dc.print_tree()

Split on Attribute 0 with Threshold 0.533076 Information Gain = 0.18547466660807876
    ├── Split on Attribute 1 with Threshold 0.639018 Information Gain = 0.34999649471050315
        ├── Split on Attribute 1 with Threshold 0.534979 Information Gain = 0.0757438417334903
            ├── Leaf Node: Predicted Label = 0
            └── Split on Attribute 0 with Threshold 0.409972 Information Gain = 0.04412566480449431
                ├── Leaf Node: Predicted Label = 0
                └── Split on Attribute 0 with Threshold 0.426073 Information Gain = 0.11134785288180327
                    ├── Split on Attribute 0 with Threshold 0.417579 Information Gain = 1.0
                        ├── Leaf Node: Predicted Label = 1
                        └── Leaf Node: Predicted Label = 0
                    └── Leaf Node: Predicted Label = 1
        └── Split on Attribute 0 with Threshold 0.111076 Information Gain = 0.18697554574592323
            ├── Split on Attribute 1 with Threshold 0.964767 Infor